In [1]:
import pyflink

In [2]:
from pyflink.table import EnvironmentSettings, TableEnvironment
from pyflink.table.expressions import col

In [10]:
table_env = TableEnvironment.create(EnvironmentSettings.in_streaming_mode())

In [11]:
table_env.execute_sql("""
    CREATE TABLE datagen (
        id INT,
        data STRING
    ) WITH (
        'connector' = 'datagen',
        'fields.id.kind' = 'sequence',
        'fields.id.start' = '1',
        'fields.id.end' = '10'
    )
""")

In [12]:
table_env.execute_sql("""
    CREATE TABLE print (
        id INT,
        data STRING
    ) WITH (
        'connector' = 'print'
    )
""")

In [14]:
r = table_env.execute_sql("show tables")

In [15]:
dir(r)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_get_java_table_schema',
 '_j_table_result',
 'collect',
 'get_job_client',
 'get_result_kind',
 'get_table_schema',
 'print',
 'wait']

In [16]:
r.print()

+------------+
| table name |
+------------+
|    datagen |
|      print |
+------------+
2 rows in set


In [17]:
s = table_env.from_path('datagen')

In [19]:
def dis(x):print(dir(x))

In [20]:
dis(s)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_j_table', '_t_env', 'add_columns', 'add_or_replace_columns', 'aggregate', 'alias', 'distinct', 'drop_columns', 'execute', 'execute_insert', 'explain', 'fetch', 'filter', 'flat_aggregate', 'flat_map', 'full_outer_join', 'get_schema', 'group_by', 'intersect', 'intersect_all', 'join', 'join_lateral', 'left_outer_join', 'left_outer_join_lateral', 'limit', 'map', 'minus', 'minus_all', 'offset', 'order_by', 'over_window', 'print_schema', 'rename_columns', 'right_outer_join', 'select', 'to_pandas', 'union', 'union_all', 'where', 'window']


In [21]:
s.to_pandas()

,id,data
0,10,59eebdb5c74601fbbf975d70f2b75ef995878272f03ed4...
1,1,c1eab9564cb91a61de1720f7b1a72038b11100ec77e800...
2,2,843feacaef1b869ac7907e297dceec5fa437bb313fd8f8...
3,3,e1ffbb4dc7d20c2e5381539eba3ae21dd714b80bd7a91b...
4,4,ebfebdfc2cf9455ab941c056108a8783a7539988d37ea3...
5,5,be2f6cad68717e77340f6b6a86e7b72088d26cd8503977...
6,6,b87d1d7d294689489f5d5e9580f9415284e35e3ffa55e8...
7,7,f1a98ab5799a63da87b8ffaa91dc2a61256ddef1abef38...
8,8,2c2d4caa8f6cce5da545d27c92088c067d6c72c466a063...
9,9,f22ee8b7c8703f0357dbaa21cc2ad9473dd5940e107477...


In [22]:
s.print_schema()

(
  `id` INT,
  `data` STRING
)


In [23]:
type(s)

pyflink.table.table.Table

In [27]:
s.distinct().to_pandas()

,id,data
0,9,8e380e27af1402143f0464b9ffd66218afb2acdfb86b8b...
1,5,1bbbec6028a92848728ed942acd1cde7df02201a42918b...
2,2,60286a8d2887804b8919e13a4fedf7bc37d1fd4821725c...
3,6,7bab6c16dbdc69edabe9b45f41cda9637a4a5b7692bffa...
4,1,37a888dc2e3e6550f30535b6c237c56729ca2c8888b2ef...
5,4,abffdd52ab8de73526cfcd3a446c0aaa67fb35fefe47d9...
6,3,0e3cecb7e80c8811e78ab2bb8b31b0b793482621c1719b...
7,7,d1eb59b53b0eac4c62ebca73c0d40bb19fe5800631ceea...
8,8,6a2fae820e23d60089500b4a4424f78d71c30f450cbe2e...
9,10,43e9d9e31e51e8334cb5d2d2a8be8ffa9932f4f677214e...


In [29]:
print(s.explain())

== Abstract Syntax Tree ==
LogicalTableScan(table=[[default_catalog, default_database, datagen]])

== Optimized Physical Plan ==
TableSourceScan(table=[[default_catalog, default_database, datagen]], fields=[id, data])

== Optimized Execution Plan ==
TableSourceScan(table=[[default_catalog, default_database, datagen]], fields=[id, data])



In [30]:
s.select(col('id') + 1, col('data')).execute_insert('print').wait()

7> +I[8, 202ee80a1192073a317f0fd220ab662b8b1f354a9825c490ebe4cb015a9fd1d6bd7b18fd6f928460c0c8eafff04f46ade4d4]
6> +I[7, 41b94a9e6092f5aadc08093b106957c6477a972da6c376f0958ec1305f069a2632f60c1bb09d8cbce86371e1de7bcf49111a]
10> +I[11, 0e3f2c68b93f8b94df39f20d523f2ab9c47a99a40105238fbdd568a122feda65318d5c34a17c8f57344137253c6ed23ea5cb]
4> +I[5, 362eea17594f21892869719e10908f1b5cf8e3163a1314424635e04d94b19d8f8ff1c74c22b6c10ac7d040562ff5495d39e9]
8> +I[9, 946f991609ac5961f48a6e3fda31078b6844ca83b6f0361d68402659569350f48ada393df49ff2aad96e2ec78ecc5688e847]
9> +I[10, b62e0aadf11fe30031b9b6e5eac0cf4bd009e4473c2e2e9271aa980ee3e2bc6f208887821403fa24762eaa04af65eed4487c]
5> +I[6, 93bc886bb51f376b40a036cd05be65b36ec1ffaa568a7200d969149c85b8460c08575512b14d57978e76d13d8393760c7236]
3> +I[4, a27c5b418693648957075193053fb50bc0eaf1339cc0aa3086008019436749cae566f76a59fd30e464a36d86b508f0e4a9b6]
1> +I[2, d86d1dc33148352ea4fa66899530c614c14cac7292d0bd77f20ca22e19244c047fd0d756c92b8c486a85e6955919458c4579

In [31]:
env = table_env

In [35]:
env.from_elements([['a', 'b']], ['c','d']).execute().print()

+----+--------------------------------+--------------------------------+
| op |                              c |                              d |
+----+--------------------------------+--------------------------------+
| +I |                              a |                              b |
+----+--------------------------------+--------------------------------+
1 row in set


In [36]:
s.get_schema()

root
 |-- id: INT
 |-- data: STRING

In [37]:
type(s)

pyflink.table.table.Table

In [38]:
dis(s)

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_j_table', '_t_env', 'add_columns', 'add_or_replace_columns', 'aggregate', 'alias', 'distinct', 'drop_columns', 'execute', 'execute_insert', 'explain', 'fetch', 'filter', 'flat_aggregate', 'flat_map', 'full_outer_join', 'get_schema', 'group_by', 'intersect', 'intersect_all', 'join', 'join_lateral', 'left_outer_join', 'left_outer_join_lateral', 'limit', 'map', 'minus', 'minus_all', 'offset', 'order_by', 'over_window', 'print_schema', 'rename_columns', 'right_outer_join', 'select', 'to_pandas', 'union', 'union_all', 'where', 'window']


In [40]:
d = env.from_elements([['a', 'b']], ['c1', 'c2'])
type(d)

pyflink.table.table.Table

In [43]:
env.create_temporary_view('tmp', d)

In [44]:
env.from_path('tmp').execute().print()

+----+--------------------------------+--------------------------------+
| op |                             c1 |                             c2 |
+----+--------------------------------+--------------------------------+
| +I |                              a |                              b |
+----+--------------------------------+--------------------------------+
1 row in set


In [45]:
orders = table_env.from_elements([('Jack', 'FRANCE', 10), ('Rose', 'ENGLAND', 30), ('Jack', 'FRANCE', 20)],
                                 ['name', 'country', 'revenue'])


In [57]:
from pyflink.table.expressions import call

new_order = orders.select(col('name'), col('revenue')).group_by(col('name')).select(col('name'), call('sum', col('revenue')).alias('res'))
new_order.execute().print()

+----+--------------------------------+----------------------+
| op |                           name |                  res |
+----+--------------------------------+----------------------+
| +I |                           Rose |                   30 |
| +I |                           Jack |                   10 |
| -U |                           Jack |                   10 |
| +U |                           Jack |                   30 |
+----+--------------------------------+----------------------+
4 rows in set


In [49]:
env.execute_sql('select * from tmp').print()

+----+--------------------------------+--------------------------------+
| op |                             c1 |                             c2 |
+----+--------------------------------+--------------------------------+
| +I |                              a |                              b |
+----+--------------------------------+--------------------------------+
1 row in set


In [51]:
res = env.execute_sql('select * from tmp')
for r in res.collect():
    print(r)

<Row('a', 'b')>


In [53]:
orders.to_pandas()

,name,country,revenue
0,Jack,FRANCE,10
1,Rose,ENGLAND,30
2,Jack,FRANCE,20


In [54]:
type(orders)

pyflink.table.table.Table

In [58]:
print(new_order.explain())

== Abstract Syntax Tree ==
LogicalProject(name=[$0], res=[$1])
+- LogicalAggregate(group=[{0}], EXPR$0=[SUM($1)])
   +- LogicalProject(name=[$0], revenue=[$2])
      +- LogicalTableScan(table=[[*anonymous_python-input-format$10*]])

== Optimized Physical Plan ==
GroupAggregate(groupBy=[name], select=[name, SUM(revenue) AS EXPR$0])
+- Exchange(distribution=[hash[name]])
   +- Calc(select=[name, revenue])
      +- TableSourceScan(table=[[*anonymous_python-input-format$10*]], fields=[name, country, revenue])

== Optimized Execution Plan ==
GroupAggregate(groupBy=[name], select=[name, SUM(revenue) AS EXPR$0])
+- Exchange(distribution=[hash[name]])
   +- Calc(select=[name, revenue])
      +- TableSourceScan(table=[[*anonymous_python-input-format$10*]], fields=[name, country, revenue])



In [62]:
stat = env.create_statement_set()

stat.add_insert('print', s)

print(stat.explain())

== Abstract Syntax Tree ==
LogicalSink(table=[default_catalog.default_database.print], fields=[id, data])
+- LogicalTableScan(table=[[default_catalog, default_database, datagen]])

== Optimized Physical Plan ==
Sink(table=[default_catalog.default_database.print], fields=[id, data])
+- TableSourceScan(table=[[default_catalog, default_database, datagen]], fields=[id, data])

== Optimized Execution Plan ==
Sink(table=[default_catalog.default_database.print], fields=[id, data])
+- TableSourceScan(table=[[default_catalog, default_database, datagen]], fields=[id, data])

